In [9]:


import tensorrt as trt
import numpy as np
import time
import torch
import os 

class Engine123:
    def __init__(self):
        pass

    def load_engine(self, runtime,engine_path):
        trt.init_libnvinfer_plugins(None, "")             
        with open(engine_path, 'rb') as f:
            engine_data = f.read()
        engine = runtime.deserialize_cuda_engine(engine_data)
        return engine
 
    
    def make_context(self,trt_engine_path):
        logger = trt.Logger(trt.Logger.WARNING)
        runtime = trt.Runtime(logger)

        self.engine = self.load_engine(runtime,trt_engine_path)
        print('gggg')
        self.context = self.engine.create_execution_context()

        self.input_shape = self.engine.get_profile_shape(0,0)[2]
        self.out_shape = self.engine.get_binding_shape(1)
        print(self.input_shape,self.out_shape)
    
    def do_inference_v2(self, input_data):
        img_batch = input_data.shape[0]  

        self.out_shape[0] = img_batch
        self.input_shape[0] = img_batch

        output_data = torch.empty(size=tuple(self.out_shape), dtype=torch.float32, device=torch.device("cuda:0"))

        self.context.set_binding_shape(0, tuple(self.input_shape))             

        bindings = None     

        bindings = [
            int(input_data.contiguous().data_ptr())
            ,int(output_data.data_ptr()) 
        ] 

        self.context.execute_async_v2(bindings,stream_handle=torch.cuda.current_stream().cuda_stream)   

        return output_data
      
        
    
trt_engine_path = './dnn_128.trt'


engine = Engine123()      
engine.make_context(trt_engine_path)
# #서있음
# pose = [[ 96.080215 , 57.818184],
#  [101.181816  ,57.818184],
#  [ 97.78075   ,56.11765 ],
#  [104.582886  ,68.02139 ],
#  [119.8877    ,73.12299 ],
#  [ 84.176476  ,90.12834 ],
#  [128.39038  ,103.73262 ],
#  [ 50.165775 , 59.51872 ],
#  [159.       ,125.83958 ],
#  [ 21.256685 , 22.106953],
#  [141.99466  , 98.63102 ],
#  [ 75.6738   ,178.55615 ],
#  [106.283424 ,183.65776 ],
#  [ 50.165775 ,229.57219 ],
#  [118.187164 ,234.6738  ],
#  [ 58.66845  ,299.29413 ],
#  [141.99466  ,300.99466 ]]

# 누워있음
pose = [[ 297.92807,   -128.51799  ],
  [ 356.34534    ,335.8993   ],
  [ 213.22302    ,271.6403   ],
  [ 204.46043    ,-67.179855 ],
  [   8.76259   ,-151.8849   ],
  [  58.417267  ,  14.604317 ],
  [ 321.29498   , -87.6259   ],
  [  -1.4604317 , 381.17267  ],
  [  55.496407  , 110.99281  ],
  [ 210.30217   , -26.287771 ],
  [ 385.554     , -93.46763  ],
  [ 116.83453   ,-148.96404  ],
  [ 330.05756   , 137.28058  ],
  [ 248.27339   , 370.94965  ],
  [ 242.43166   , 125.59713  ],
  [ 338.82016   , -32.129498 ],
  [ 214.68346   ,-162.10793  ]]
from sklearn.preprocessing import MinMaxScaler


def _minMaxScaling(data):
    scaler = MinMaxScaler()
    scaler.fit(data)
    return scaler.transform(data)  


pose = np.array(pose)

s = time.time()
pose = _minMaxScaling(pose)  
# print('time',time.time()-s)
# print('pose',pose)
pose = torch.tensor(pose).to(dtype=torch.float32)   
print('pose.shape',pose.shape)

result = torch.empty([pose.shape[0], pose.shape[1]], dtype=torch.float32, device=torch.device("cuda:0"))

print('result.shape',result.shape)
# for i in range(5):
#     if i == 1 :
#         None
    

pose_list = [result for _ in range(1)]

pose = torch.stack(pose_list,dim=0)
pose = torch.tensor(pose).cuda() 
# print(pose.shape)  


output_data = engine.do_inference_v2(pose) #결과 생성
# print(output_data)
# sm = nn.Softmax(doutput_data)
# print(sm)
pred = output_data.max(1, keepdim=True)
print(pred)



gggg
(128, 17, 2) (-1, 2)
pose.shape torch.Size([17, 2])
result.shape torch.Size([17, 2])
torch.return_types.max(
values=tensor([[23.6406]], device='cuda:0'),
indices=tensor([[1]], device='cuda:0'))


[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0
[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
print(1.0000e+00)

In [ ]:
# zip은 튜플 형태를 반환하므로 (a,n)으로 묶어준다
for i,(a,n) in enumerate(zip(alpha,num)):
    print(i,a,n)